Hello Everyone !! This is going to my learning from other kernels

We have to identify author of the given sentence. This is multiclass classification problem


The competition dataset contains text from works of fiction written by spooky authors of the public domain:

* Edgar Allan Poe (EAP)
* HP Lovecraft (HPL)
* Mary Wollstonecraft Shelley (MWS)

First, lets load all the libraries needed

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plotting
import seaborn as sns # plotting
%matplotlib inline

import string

#plotly related 
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
py.init_notebook_mode(connected = True)

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import sent_tokenize,word_tokenize
from collections import Counter

import scipy

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import ensemble, metrics, model_selection, naive_bayes
from sklearn.decomposition import TruncatedSVD
import xgboost as xgb
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

***Helper Functions***
These function will be useful in the later steps

In [ ]:
#function to display plots
def displayPlots(type='bar',x=None, y=None, data=None, hue=None, color=None, title=None, figsize = (11,8), rotate = 0):
    fig, ax = plt.subplots(figsize = figsize)
    if type == 'bar':
        ax = displySnsBarPlot(x=x, y=y, data =data,hue=hue, color = color)
    elif type == 'violin':
        ax = displaySnsViolinPlot(x=x, y=y, data =data,hue=hue, color = color)
    elif type == 'box':
        ax = displaySnsBoxPlot(x=x, y=y, data =data,hue=hue, color = color)
    elif type == 'strip':
        ax = displySnsStripPlot(x=x, y=y, data =data,hue=hue, color = color)
    plt.xticks(rotation = 45)
    
def displySnsBarPlot(x=None, y=None, data=None, hue=None, color=None):
    sns.barplot(x =x, y=y, data =train, hue = hue, color=color)

def displaySnsViolinPlot(x=None, y=None, data=None, hue=None, color=None):
    sns.violinplot(x =x, y=y, data =train, hue = hue, color=color)
    
def displaySnsBoxPlot(x=None, y=None, data=None, hue=None, color=None):
    sns.boxplot(x =x, y=y, data =train, hue = hue, color=color)

def displySnsStripPlot(x=None, y=None, data=None, hue=None, color=None):
    sns.stripplot(x =x, y=y, data =train, hue = hue, color=color, jitter =True)

def displayWordCloud(data = None, backgroundcolor = 'black',  width=800, height=600 ):
    wordcloud = WordCloud(stopwords = STOPWORDS, background_color = backgroundcolor, 
                         width = width, height = height).generate(data)
    plt.figure(figsize = (15 , 10))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()
    
def runNM(train_X, train_y, test_X, test_y, test_X2 ):
    print(train_X)
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_text_y = model.predict_proba(test_X)
    pred_text_y2 = model.predict_proba(test_X2)
    return pred_text_y, pred_text_y2, model

def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.3):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 3
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = child
    param['subsample'] = 0.8
    param['colsample_bytree'] = colsample
    param['seed'] = seed_val
    num_rounds = 2000

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest, ntree_limit = model.best_ntree_limit)
    if test_X2 is not None:
        xgtest2 = xgb.DMatrix(test_X2)
        pred_test_y2 = model.predict(xgtest2, ntree_limit = model.best_ntree_limit)
    return pred_test_y, pred_test_y2, model
    

In [ ]:
# loading train and test dataset
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")


In [ ]:
# lets look at the few entries in train and test dataset
print(train.head())
print(test.head())

In [ ]:
#Checking for missing values in train and test dataset
print(train.isnull().sum())
print(test.isnull().sum())

In [ ]:
#there are no missing value in both training and test data
#lets see how data are distributed among authors in training data
displayPlots(type = 'bar', x =train.author.unique(), y = train.author.value_counts().values, title = "Distribution of words among authors" )

In [ ]:
#given dataset is quite balanced
#lets see some of the sentences writtern by each author
train_eap = train[train["author"] == 'EAP']["text"]
train_hpl = train[train["author"] == 'HPL']["text"]
train_mws = train[train["author"] == 'MWS']["text"]

print("EAP")
print(train_eap.sample(3).values,'\n')
print("HPL")
print(train_hpl.sample(3).values,'\n')
print("MWS")
print(train_mws.sample(3).values,'\n')

In [ ]:
#display all words using wordcloud
displayWordCloud(' '.join(train['text']))

In [ ]:
#display words used by EAP using wordcloud
displayWordCloud(' '.join(train_eap))

In [ ]:
displayWordCloud(' '.join(train_hpl))

In [ ]:
displayWordCloud(' '.join(train_mws))

In [ ]:

# calculate different measures of the quantity of information
train['sentences'] = train.text.transform(lambda x: len(sent_tokenize(x)))
train['words'] = train.text.transform(lambda x: len(word_tokenize(x)))
train['text_len'] = train.text.transform(lambda x: len(x))

test['sentences'] = test.text.transform(lambda x: len(sent_tokenize(x)))
test['words'] = test.text.transform(lambda x: len(word_tokenize(x)))
test['text_len'] = test.text.transform(lambda x: len(x))

train.groupby("author")[['sentences','words','text_len']].sum().plot.bar(subplots = True, layout=(1,3), figsize=(16,8))

In [ ]:
#length of unique words
train["unique_words"] = train["text"].apply(lambda x : len(set(x.split())))
test["unique_words"] = test["text"].apply(lambda x : len(set(x.split())))
#length of stop words
eng_stopwords = set(stopwords.words("english"))
train["stop_words"] = train["text"].apply(lambda x : len([w for w in str(x).split() if w in eng_stopwords]))
test["stop_words"] = test["text"].apply(lambda x : len([w for w in str(x).split() if w in eng_stopwords]))

#length of punctuations
train["punctuations"] = train["text"].apply(lambda x : len([w for w in str(x) if w in string.punctuation]))
test["punctuations"] = test["text"].apply(lambda x : len([w for w in str(x) if w in string.punctuation]))

#upper case letters
train["upper_case"] = train["text"].apply(lambda x : len([w for w in str(x).split() if w.isupper()]))
test["upper_case"] = test["text"].apply(lambda x : len([w for w in str(x).split() if w.isupper()]))

#title letters
train["Title"] = train["text"].apply(lambda x : len([w for w in str(x).split() if w.istitle()]))
test["Title"] = test["text"].apply(lambda x : len([w for w in str(x).split() if w.istitle()]))

train.groupby('author')[['unique_words' , 'stop_words' , 'punctuations' , 'upper_case']].sum().plot.bar(subplots = True, layout = (1,4), figsize = (16,8))




Lets create base model and check how well it is performing using the features which we have generated in the above steps

In [ ]:
train_y = train.author.map({'EAP' : 0, 'HPL' : 1, 'MWS' : 2})


In [ ]:
vect = CountVectorizer(ngram_range=(1,2),min_df = 5).fit(train['text'])
train_vectorized = vect.transform(train['text'])
len(vect.get_feature_names())

In [ ]:
stack = train[['sentences' , 'words' , 'text_len' ]]
train_vectorized = scipy.sparse.hstack([train_vectorized,stack])
model = LogisticRegression(C=.03,n_jobs = -1)
model.fit(train_vectorized , train['author'])

In [ ]:
test_vectorized = vect.transform(test['text'])
stack1 = test[['sentences' , 'words' , 'text_len' ]]
test_vectorized = scipy.sparse.hstack([test_vectorized,stack1])


In [ ]:
df = pd.DataFrame(model.predict_proba(test_vectorized), index= test['id'] , columns = ['EAP','HLP','MWS'])
df.head()

In [ ]:
tfidf_vec = TfidfVectorizer(stop_words = 'english', ngram_range = (1,3))
full_tfidfvet = tfidf_vec.fit_transform(train["text"].values.tolist() + test["text"].values.tolist())
train_tfidfvec = tfidf_vec.fit_transform(train["text"].values.tolist())
test_tfidfvec = tfidf_vec.fit_transform(test["text"].values.tolist())

In [ ]:
n_comp = 20
svd_obj = TruncatedSVD(n_components = n_comp, algorithm = 'arpack')
svd_obj.fit(full_tfidfvet)
train_svd = pd.DataFrame(svd_obj.fit_transform(train_tfidfvec))
test_svd = pd.DataFrame(svd_obj.fit_transform(test_tfidfvec))

train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
train  = pd.concat([train , train_svd], axis = 1)
test  = pd.concat([test , test_svd], axis = 1)

del full_tfidfvet, train_tfidfvec, test_tfidfvec, train_svd, test_svd

In [ ]:
cols_to_drop = ['id', 'text']

train_X = train.drop(cols_to_drop + ['author'], axis = 1)
test_X = test.drop(cols_to_drop, axis = 1)

kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train.shape[0], 3])
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_X.loc[dev_index], train_X.loc[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runXGB(dev_X, dev_y, val_X, val_y, test_X, seed_val=0)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    break
print("cv scores : ", cv_scores)

In [ ]:
fig, ax = plt.subplots(figsize = (12,12))
xgb.plot_importance(model, max_num_features = 50, height = 0.8, ax=ax)
plt.show()

In [ ]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 1007)

In [ ]:
classifier.fit(train_X, train_y)
y_pred = classifier.predict(test_X)